<a href="https://colab.research.google.com/github/Shazizan/pyspark/blob/main/ApacheSpark_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="images/cads-logo.png" style="height: 100px;padding-top:5px" align=left> <img src="images/apache_spark.png" style="height: 20%;width:20%; padding-top:0px" align=right>

# Manipulating Data using Apache Spark

In this notebook, we are going to get our hands dirty with Spark DataFrame API to perform common data operations.

In [ ]:
! pip install pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

     |████████████████████████████████| 204.2MB 66kB/s 
     |████████████████████████████████| 204kB 27.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=59006a5e083ff1221cdd57bf93b6d855416456101df07cc4926e5deed0b9ed8e
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


If you take a look at dataset folder, you will see `flights.csv` that contains a row for every flight that left Portland International Airport (PDX) or Seattle-Tacoma International Airport (SEA) in 2014 and 2015.

In the first step, we should create a DataFrame using `flights.csv` file and then create a table (temporaray view) for querying flights by using SQL commands. Let's do it.

In [ ]:
import os
MAIN_DIRECTORY = os.getcwd()
file_path =MAIN_DIRECTORY+"/dataset/flights.csv"

In [ ]:
df_flights = spark.read.format("csv").option("header","true").option('inferSchema','true').load(file_path)

In [ ]:
# A simple way to create a dataframe in Spark
# df_flights = spark.read.csv(file_path, header=True, inferSchema = True)

In [ ]:
df_flights.createOrReplaceTempView('flights')

### Exercise 1: Use SQL to get the first five rows of the flights table and save the result to flights5, finally show the results.

In [ ]:
flights5 = spark.sql("SELECT * FROM flights LIMIT 5")
flights5.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
+----+-----+---+--------+---------+-----

### Exercise 2: Write a query that counts the number of flights to each airport from SEA and PDX.

In [ ]:
spark.sql("SELECT origin, dest, count(*) FROM flights GROUP BY origin , dest").show()

+------+----+--------+
|origin|dest|count(1)|
+------+----+--------+
|   SEA| RNO|       8|
|   SEA| DTW|      98|
|   SEA| CLE|       2|
|   SEA| LAX|     450|
|   PDX| SEA|     144|
|   SEA| BLI|       5|
|   PDX| IAH|      57|
|   PDX| PHX|     209|
|   SEA| SLC|     225|
|   SEA| SBA|      23|
|   SEA| BWI|      29|
|   PDX| IAD|      23|
|   PDX| SFO|     305|
|   SEA| KOA|      40|
|   PDX| MCI|      15|
|   SEA| SJC|     213|
|   SEA| ABQ|      43|
|   SEA| SAT|      18|
|   PDX| ONT|      57|
|   SEA| LAS|     364|
+------+----+--------+
only showing top 20 rows



In [ ]:
df_flights.groupBy(['origin','dest']).count().show()

+------+----+-----+
|origin|dest|count|
+------+----+-----+
|   SEA| RNO|    8|
|   SEA| DTW|   98|
|   SEA| CLE|    2|
|   SEA| LAX|  450|
|   PDX| SEA|  144|
|   SEA| BLI|    5|
|   PDX| IAH|   57|
|   PDX| PHX|  209|
|   SEA| SLC|  225|
|   SEA| SBA|   23|
|   SEA| BWI|   29|
|   PDX| IAD|   23|
|   PDX| SFO|  305|
|   SEA| KOA|   40|
|   PDX| MCI|   15|
|   SEA| SJC|  213|
|   SEA| ABQ|   43|
|   SEA| SAT|   18|
|   PDX| ONT|   57|
|   SEA| LAS|  364|
+------+----+-----+
only showing top 20 rows



### Exercise 3: Write a piece of code to create a DataFrame using `airports.csv`, this file contains information about different airports all over the world.

In [ ]:
file_path = MAIN_DIRECTORY +'/dataset/airports.csv'
df_airports = spark.read.csv(file_path, header=True, inferSchema=True)
df_airports.show(10)

+---+--------------------+----------+------------+----+---+---+
|faa|                name|       lat|         lon| alt| tz|dst|
+---+--------------------+----------+------------+----+---+---+
|04G|   Lansdowne Airport|41.1304722| -80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|32.4605722| -85.6800278| 264| -5|  A|
|06C| Schaumburg Regional|41.9893408| -88.1012428| 801| -6|  A|
|06N|     Randall Airport| 41.431912| -74.3915611| 523| -5|  A|
|09J|Jekyll Island Air...|31.0744722| -81.4277778|  11| -4|  A|
|0A9|Elizabethton Muni...|36.3712222| -82.1734167|1593| -4|  A|
|0G6|Williams County A...|41.4673056| -84.5067778| 730| -5|  A|
|0G7|Finger Lakes Regi...|42.8835647| -76.7812318| 492| -5|  A|
|0P2|Shoestring Aviati...|39.7948244| -76.6471914|1000| -5|  U|
|0S9|Jefferson County ...|48.0538086|-122.8106436| 108| -8|  A|
+---+--------------------+----------+------------+----+---+---+
only showing top 10 rows



Let's look at performing column-wise operations. In Apache Spark, you can do this using the `.withColumn(colName, col)`  which returns a new DataFrame by adding a column or replacing the existing column that has the same name.

*Parameters*:  
- **colName** – string, name of the new column.
- **col** – a Column expression for the new column.

The new `column` must be an object of class Column. Creating one of these is as easy as extracting a column from your DataFrame using `df.colName`.
Apache Spark DataFrame is **immutable**. Immutable means that it can't be changed, and so columns can't be updated in place.
For example:
```python
df = df.withColumn("newCol", df.oldCol + 1)
```
The above code creates a DataFrame with the same columns as df plus a new column, `newCol`, where every entry is equal to the corresponding entry from `oldCol`, plus one.

Sometimes we have to change a column data type to another one, in this case, we can use the following code:
```python
from pyspark.sql.functions import col
df_name = df_name.withColumn("columnName", col("columnName").cast("DataType"))
```

### Exercise 4: Update `flights` DataFrame to include a new column called `duration_hrs`, that contains the duration of each flight in hours.

### Exercise 5: Write a query using the `.filter()` method to find all the flights that flew over 1000 miles.

### Exercise 6: Write a query using `.filter()` method, to only keep flights from SEA to PDX. This query should only return `tailnum`, `origin`, and `dest` columns.

In [ ]:
# Solution 1


In [ ]:
#Solution 2


We can perform column-wise operations using `.select()` method. When we select a column using the `df.colName` notation. In `.select()` method, we can perform any column operation and it will return the transformed column.
For example, the following command returns a column of flight durations in hours instead of minutes.
```python
df_flights.select(df_flights.air_time/60)
```
We can use the `alias()` method to rename a column we've selected. The following example shows how we can do that.
```python
df_flights.select((df_flights.air_time/60).alias("duration_hrs")
```
If we want to stick to the SQL syntax, we can use `.selectExpr()` method as well. The following commad is equivalent to the previous code.

```python
df_flights.selectExpr("air_time/60 as duration_hrs")
```

### Exercise 7: Write a query that return these columns, `origin`, `dest`, `tailnum`, and average speed in KM per hour.

In [ ]:
#Solution 1


In [ ]:
#Solution 2


### Exercise 8: Find the the shortest (in terms of distance) flight that left PDX by first filtering and using the `.min()` method. Perform the filtering by referencing the column directly, not passing a SQL string.

In [ ]:
#Solution 1


In [ ]:
#Solutin 2


In [ ]:
# Solution 3


### Exercise 9: Find the the longest (in terms of time) flight that left SEA by filtering and using the `.max()` method. Perform the filtering by referencing the column directly, not passing a SQL string.

if we run the following code, we will get an error, because `air_time` data type is string, first we should cast it to an integer column.

In [ ]:
#Solution 1


In [ ]:
#Solution 2


### Exercise 10: Write a query that uses the `.avg()` method to get the average air time of Delta Airlines flights ( the carrier column value is "DL") that left SEA.

### Exercise 11: Write a query that uses the `.sum()` method to get the total number of hours all planes spent in the air by creating a column called `duration_hrs` from the column `air_time`.

### Exercise 12: Write a query that uses `tailnum` column to count the number of flights each plane made.

### Exercise 13: Write a query that returns the average duration of flights from `PDX` and `SEA`.

### Exercise 14: Write a query that returns the average departure delay (`dep_delay`) in each month for each destination. Then import PySpark functions to calculate the standard deviation of `dep_delay` by using `stddev()` function.

### Exercise 15: Write a query that performs left outer join on the flights and airports DataFrames.
- The flights and airports DataFrames are already in the workspace.
- First, examine the airports DataFrame by calling .show() method.
- Note which key column will let you join these two DataFrames.
- Before joining these two DataFrames, rename the `faa` column in `airports` to `dest`, and then convert this DataFrame to a temporary view (table).
- Use `spark.sql` to perform left outer join on these two tables.


### Exercise 16: Rewrite the previous query by using DataFrame API `.join` method.

In PySpark, we can use `.join` method to perform joins. This method takes three arguments.
- The first argument is the second DataFrame that we want to join with the first one.
- The second argument, `on`, is the name of the key column(s) as a string. The names of the key column(s) must be the same in each table.
- The third argument, `how`, specifies the kind of join to perform.

To perform left outer join set the value of `how` to `"leftouter"`.

#### Awesome